In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os
os.chdir("D:/Quarter3/QAM/Project/My project")
from math import sqrt

In [2]:
Industry = pd.read_csv('10_Industry_Portfolios.csv',skiprows=11)
Industry = Industry.iloc[0:1126,:]

FF_mkt = pd.read_csv('F-F_Research_Data_Factors.csv',skiprows =3)
FF_mkt = FF_mkt.iloc[0:1126,:]

In [3]:
# Cleaning Industry data
ind_data = Industry.copy()
ind_data['Date'] = pd.to_datetime((ind_data.iloc[:,0].astype(str) + '01'),format="%Y%m%d") 
ind_data['Year'] = pd.DatetimeIndex(ind_data.loc[:,'Date']).year
ind_data['Month'] = pd.DatetimeIndex(ind_data.loc[:,'Date']).month
ind_data = ind_data.loc[((ind_data['Year']>=1989) & (ind_data['Year']<=2019)),:].reset_index(drop=True)
ind_data = ind_data.iloc[8:372,:].reset_index(drop=True)
ind_data

,Unnamed: 0,NoDur,Durbl,Manuf,Enrgy,HiTec,Telcm,Shops,Hlth,Utils,Other,Date,Year,Month
0,198909,-0.54,-4.60,-2.53,1.41,-2.71,3.81,-2.23,2.38,0.48,1.42,1989-09-01,1989,9
1,198910,-2.49,-5.70,-4.36,0.46,-4.42,-2.04,-4.11,0.39,1.01,-4.85,1989-10-01,1989,10
2,198911,1.68,-1.18,3.06,3.93,-1.80,1.99,1.36,4.53,3.23,-0.02,1989-11-01,1989,11
3,198912,1.36,-1.45,2.99,5.86,-1.09,5.73,1.24,1.05,4.79,-1.50,1989-12-01,1989,12
4,199001,-9.43,-3.84,-6.29,-4.14,-1.28,-13.40,-6.24,-7.37,-5.35,-8.83,1990-01-01,1990,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
359,201908,-1.28,-4.24,-1.83,-8.83,-2.79,0.43,0.11,-0.45,3.55,-4.39,2019-08-01,2019,8
360,201909,1.81,3.26,4.02,3.69,1.14,1.30,0.86,-0.54,3.92,1.98,2019-09-01,2019,9
361,201910,0.01,7.14,0.12,-2.05,3.35,2.23,0.70,4.47,-1.39,2.95,2019-10-01,2019,10
362,201911,2.23,2.80,3.35,1.23,5.11,1.75,2.20,5.30,-2.07,4.96,2019-11-01,2019,11


In [4]:
# Cleaning Market data 
ff_data = FF_mkt.iloc[:,[0,1,4]].copy()
ff_data['Date'] =  pd.to_datetime((ff_data.iloc[:,0].astype(str) + '01'),format="%Y%m%d")
ff_data['Year'] = pd.DatetimeIndex(ff_data.loc[:,'Date']).year
ff_data['Month'] = pd.DatetimeIndex(ff_data.loc[:,'Date']).month
ff_data = ff_data.loc[((ff_data['Year']>=1989) & (ff_data['Year']<=2019)),:].reset_index(drop=True)
ff_data = ff_data.iloc[8:372,:].reset_index(drop=True)
ff_data

,Unnamed: 0,Mkt-RF,RF,Date,Year,Month
0,198909,-0.76,0.65,1989-09-01,1989,9
1,198910,-3.67,0.68,1989-10-01,1989,10
2,198911,1.03,0.69,1989-11-01,1989,11
3,198912,1.16,0.61,1989-12-01,1989,12
4,199001,-7.85,0.57,1990-01-01,1990,1
...,...,...,...,...,...,...
359,201908,-2.58,0.16,2019-08-01,2019,8
360,201909,1.44,0.18,2019-09-01,2019,9
361,201910,2.06,0.15,2019-10-01,2019,10
362,201911,3.87,0.12,2019-11-01,2019,11


In [5]:
data = pd.merge(ind_data, ff_data, on=["Unnamed: 0","Date","Year","Month"])
data = data.loc[~((data['NoDur'] == '-99.99') & (data['Durbl'] == '-99.99') & (data.Manuf == '-99.99') 
             & (data.Enrgy == '-99.99') & (data.HiTec == '-99.99') & (data.Telcm == '-99.99')
             & (data.Shops == '-99.99') & (data['Hlth '] == '-99.99') & (data.Utils == '-99.99')
             & (data.Other == '-99.99') & (data['Mkt-RF'] == '-99.99') & (data.RF == '-99.99') ),:]
data = data.iloc[:,[0,11,12,13,1,2,3,4,5,6,7,8,9,10,14,15]]
data_means = data.iloc[:,4:16]
data_means = data_means.apply(pd.to_numeric)

# New temp
temp = pd.DataFrame({"ExNoDur": data_means['NoDur'] - data_means['RF'],
                     "ExDurbl": data_means['Durbl'] - data_means['RF'], 
                     "ExManuf": data_means['Manuf'] - data_means['RF'],
                     "ExEnrgy": data_means['Enrgy'] - data_means['RF'], 
                     "ExHiTec": data_means['HiTec'] - data_means['RF'],
                     "ExTelcm": data_means['Telcm'] - data_means['RF'], 
                     "ExShops": data_means['Shops'] - data_means['RF'],
                     "ExHlth": data_means['Hlth '] - data_means['RF'], 
                     "ExUtils": data_means['Utils'] - data_means['RF'],
                     "ExOther": data_means['Other'] - data_means['RF'], 
                     "Mkt-RF":data_means['Mkt-RF']})

temp['Mean_rows'] = temp.mean(axis=1)

data = pd.concat([data.loc[:,["Date","Year","Month"]], (temp/100)],axis=1)
data

,Date,Year,Month,ExNoDur,ExDurbl,ExManuf,ExEnrgy,ExHiTec,ExTelcm,ExShops,ExHlth,ExUtils,ExOther,Mkt-RF,Mean_rows
0,1989-09-01,1989,9,-0.0119,-0.0525,-0.0318,0.0076,-0.0336,0.0316,-0.0288,0.0173,-0.0017,0.0077,-0.0076,-0.009427
1,1989-10-01,1989,10,-0.0317,-0.0638,-0.0504,-0.0022,-0.0510,-0.0272,-0.0479,-0.0029,0.0033,-0.0553,-0.0367,-0.033255
2,1989-11-01,1989,11,0.0099,-0.0187,0.0237,0.0324,-0.0249,0.0130,0.0067,0.0384,0.0254,-0.0071,0.0103,0.009918
3,1989-12-01,1989,12,0.0075,-0.0206,0.0238,0.0525,-0.0170,0.0512,0.0063,0.0044,0.0418,-0.0211,0.0116,0.012764
4,1990-01-01,1990,1,-0.1000,-0.0441,-0.0686,-0.0471,-0.0185,-0.1397,-0.0681,-0.0794,-0.0592,-0.0940,-0.0785,-0.072473
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
359,2019-08-01,2019,8,-0.0144,-0.0440,-0.0199,-0.0899,-0.0295,0.0027,-0.0005,-0.0061,0.0339,-0.0455,-0.0258,-0.021727
360,2019-09-01,2019,9,0.0163,0.0308,0.0384,0.0351,0.0096,0.0112,0.0068,-0.0072,0.0374,0.0180,0.0144,0.019164
361,2019-10-01,2019,10,-0.0014,0.0699,-0.0003,-0.0220,0.0320,0.0208,0.0055,0.0432,-0.0154,0.0280,0.0206,0.016445
362,2019-11-01,2019,11,0.0211,0.0268,0.0323,0.0111,0.0499,0.0163,0.0208,0.0518,-0.0219,0.0484,0.0387,0.026845


In [6]:
#EW 
mean_total_ew = data['Mean_rows'].mean()
sd_total_ew = data['Mean_rows'].std()
Sharpe_Ratio_total_ew =  mean_total_ew/sd_total_ew
CEQ_total_ew = mean_total_ew - 0.5*((sd_total_ew)**2)
turnover = []
turnover.insert(0,0)
temp_outsample = data.iloc[:,3:14]
for i in range(1,temp_outsample['ExNoDur'].count()):
    turnover.append(sum(abs((1/11) - ((1/11) * (1+temp_outsample.iloc[i,:]))/ (1 + sum((1/11)* temp_outsample.iloc[i,:])))))
Turnover_ew = np.mean(turnover)

In [7]:
#VW 
mean_total_vw = data['Mkt-RF'].mean()
sd_total_vw = data['Mkt-RF'].std()
Sharpe_Ratio_total_vw = mean_total_vw/sd_total_vw
CEQ_total_vw = mean_total_vw - 0.5*(sd_total_vw**2)
Turnover_vw = 0

In [8]:
#MVE in sample
sigma = np.cov(temp_outsample.T)
mu = temp_outsample.mean()
xt = np.matmul(np.linalg.inv(sigma),mu)
weights = xt/sum(xt)
ret = pd.DataFrame(weights*temp_outsample).sum(axis=1)
mean_MVE_in = ret.mean()
sd_MVE_in = ret.std()
Sharpe_MVE_in = mean_MVE_in/sd_MVE_in
CEQ_MVE_in = mean_MVE_in - 0.5*(sd_MVE_in**2)
Turnover_MVE_in = np.nan

In [9]:
#MVE Out of sample test
ret = []
w = 0
turnover = []
for i in range(60,temp_outsample['ExNoDur'].count()):
    sigma = np.cov(temp_outsample.iloc[i-60:i,:].T)
    mu = temp_outsample.iloc[i-60:i,:].mean()   
    xt = np.matmul(np.linalg.inv(sigma),mu)
    weights = xt/sum(xt)
    ret.append(weights * temp_outsample.iloc[i,:].values)
    if  (i-60 == 0):
        turnover.append(0)
    else:
        turnover.append(sum(abs(weights - (w * (1+temp_outsample.iloc[i-1,:]))/ (1 + sum(weights * temp_outsample.iloc[i-1,:])))))
    w = weights 
ret = pd.DataFrame(ret,columns = temp_outsample.columns)
ret_MVE_out = ret.sum(axis=1)
mean_MVE_out = ret_MVE_out.mean()
sd_MVE_out = ret_MVE_out.std()
Sharpe_MVE_out =  mean_MVE_out/sd_MVE_out
CEQ_MVE_out =  mean_MVE_out - 0.5*(sd_MVE_out**2)
Turnover_MVE_out = np.mean(turnover)

In [10]:
#Shrunk MVE Out of sample test
ret_shrink = []
w=0
tunrover= []
for i in range(60,temp_outsample['ExNoDur'].count()):
    returns = temp_outsample.iloc[i-60:i,:]
    T = returns['ExNoDur'].count()
    
    # Shrink covariance matrix 
    S = np.cov(returns.T)
    target= np.mean(np.diag(S))*np.eye(S.shape[1]) 
    f = lambda row: (((returns.iloc[row,:] @ returns.iloc[row,:].T) - S)**2).sum()
    omega2 = np.nanmean([f(x) for x in range(T)])/(T-1)
    total_var = sum(sum((S - target)**2))
    delta2 = total_var - omega2
    beta = max((delta2/total_var), 0)
    Sigma_hat = (1 - beta) * target + beta * S
    sigma = Sigma_hat
    
    # Shrink returns
    m_i = returns.mean()
    shrinkage_target = m_i.mean()
    #beta_denominator =  m_i.var()
    #beta_numerator = (beta_denominator - np.mean((returns.std()/sqrt(returns.shape[0]))**2)) if (beta_denominator - np.mean((returns.std()/sqrt(returns.shape[0]))**2))<0 else 0
    omega2 = (returns.var()/T).mean() if (returns.var()/T).mean()>0 else 0
    total_var = m_i.var()
    delta2 = total_var - omega2
    beta = max((delta2/total_var), 0 )
    mu = (1-beta)*shrinkage_target + beta*m_i
    xt = np.matmul(np.linalg.inv(sigma),mu)
    weights = xt/sum(xt)
    ret_shrink.append(weights * temp_outsample.iloc[i,:].values)
    if  (i-60 == 0):
        turnover.append(0)
    else:
        turnover.append(sum(abs(weights - (w * (1+temp_outsample.iloc[i-1,:]))/ (1 + sum(weights * temp_outsample.iloc[i-1,:])))))
    w = weights 
ret_shrink = pd.DataFrame(ret_shrink,columns = temp_outsample.columns)
shrink_ret_MVE_out = ret_shrink.sum(axis=1)
mean_shrink_MVE_out = shrink_ret_MVE_out.mean()
sd_shrink_MVE_out = shrink_ret_MVE_out.std()
Sharpe_shrink_MVE_out =  mean_shrink_MVE_out/sd_shrink_MVE_out
CEQ_shrink_MVE_out =  mean_shrink_MVE_out - 0.5*(sd_shrink_MVE_out**2)
Turnover_shrink_MVE_out = np.mean(turnover)

In [11]:
#Risk Parity
ret = []
w = 0
turnover = []
for i in range(60,temp_outsample['ExNoDur'].count()):
    sigma = np.cov(temp_outsample.iloc[i-60:i,:].T)
    diag_sigma = np.diag(sigma)  
    xt = 1/diag_sigma
    weights = xt/sum(xt)
    ret.append(weights * temp_outsample.iloc[i,:].values)
    if  (i-60 == 0):
        turnover.append(0)
    else:
        turnover.append(sum(abs(weights - (w * (1+temp_outsample.iloc[i-1,:]))/ (1 + sum(weights * temp_outsample.iloc[i-1,:])))))
    w = weights 
ret = pd.DataFrame(ret,columns = temp_outsample.columns)
ret_rp = ret.sum(axis=1)
mean_rp = ret_rp.mean()
sd_rp = ret_rp.std()
Sharpe_rp =  mean_rp/sd_rp
CEQ_rp =  mean_rp - 0.5*(sd_rp**2)
Turnover_rp = np.mean(turnover)

In [12]:
total = pd.DataFrame(np.array([[mean_total_ew,mean_MVE_in, mean_MVE_out, mean_shrink_MVE_out ,mean_total_vw, mean_rp],
                               [sd_total_ew, sd_MVE_in, sd_MVE_out, sd_shrink_MVE_out,sd_total_vw, sd_rp],
                               [Sharpe_Ratio_total_ew, Sharpe_MVE_in,Sharpe_MVE_out,Sharpe_shrink_MVE_out, Sharpe_Ratio_total_vw, Sharpe_rp],
                               [CEQ_total_ew, CEQ_MVE_in, CEQ_MVE_out,CEQ_shrink_MVE_out, CEQ_total_vw,CEQ_rp],
                               [Turnover_ew, Turnover_MVE_in, Turnover_MVE_out, Turnover_shrink_MVE_out, Turnover_vw, Turnover_rp]]),
                    columns = ["EW","MVE Insample","MVE Outsample","Shrunk MVE Outsample","VW","RP"])#,
                    #rows = ["Mean","SD","Sharpe Ratio","CEQ","Turnover"])

In [13]:
total = total.T

In [14]:
total.columns = ["Mean","SD","Sharpe Ratio","CEQ","Turnover"]

In [15]:
total

,Mean,SD,Sharpe Ratio,CEQ,Turnover
EW,0.006819,0.039300,0.173520,0.006047,0.021944
MVE Insample,0.012744,0.044141,0.288708,0.011770,NaN
MVE Outsample,0.021820,0.369255,0.059091,-0.046355,26.542581
Shrunk MVE Outsample,0.005613,0.047135,0.119092,0.004503,13.409707
VW,0.006558,0.042175,0.155488,0.005668,0.000000
RP,0.007303,0.036751,0.198709,0.006627,0.028346
